In [4]:
import re
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import numpy
import csv

import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import multi_gpu_model



def garbage_remove(tconst):
    train = pd.read_csv(r'./Horror set/Horror.csv', skiprows=1, names=['Text', 'sentiment'])
    test = pd.read_csv(r"./Web_Crowling/" + tconst + ".tsv",sep='\t', names=['Rank', 'ID', 'Title','Text','sentiment'])
    print(tconst)
    A, B = [], []

    for i in train['Text']:
        i = re.sub('[^a-zA-Z0-9 \n]', '', str(i))
        i = i.replace("br","").replace("  "," ").replace("   "," ")
        A.append(i)

    for i in test['Text']:
        i = re.sub('[^a-zA-Z0-9 \n]', '', str(i))
        i = i.replace("br","").replace("  "," ").replace("   "," ")
        B.append(i)

    train['Text'] = A
    test['Text'] = B
    return train,test


#쓰레기 값을 제거한 데이터들에 대해서, 백터화-웟 핫 인코딩을 한 뒤, 반환합니다.
def preprocessing(garbage_remove):
    t = Tokenizer(num_words=1250)
    train,test = garbage_remove
    test_label, train_label = [], []

    t.fit_on_texts(train['Text'])
    t.fit_on_texts(test['Text'])

    #cruel = 1, astonish = 0 로 설정합니다.
    for i in train['sentiment']:
        if i == 'cruel':
            train_label.append(1)
        else:
            train_label.append(0)
    
    #(이 부분 고려 필요) test_label 을 모두 cruel 로 설정, 정확도가 떨어질수록 astonish에 가깝다는 논지로, 상당히 정확합니다.
    # 다만 .h5파일로 저장하여 불러들인뒤, 다른 데이터 셋(B)을 추가하여 확인하면
    # 다른 데이터셋 (B)를 Test 데이터로 둔 것보다 훨신 떨어지는 결과를 얻습니다.
    for i in test['sentiment']:
            test_label.append(1)

    train_result = t.texts_to_sequences(train['Text'])
    test_result = t.texts_to_sequences(test['Text'])

    # 원 핫 인코딩
    train_result = t.sequences_to_matrix(train_result, mode='binary')
    test_result = t.sequences_to_matrix(test_result, mode='binary')
    return train_result, test_result, train_label, test_label


tconst = "tt4361050"
X_train,X_test,y_train,y_test = preprocessing(garbage_remove(tconst))

top_words = 2000
max_words = 1250

model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

early_stopping = EarlyStopping(monitor='acc', min_delta=0, patience=3, verbose=0, mode='auto') # 조기종료 콜백함수 정의
hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=128, verbose=1, callbacks=[early_stopping])

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


from keras.models import load_model
model.save('temp2.h5')





tt4361050
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1250, 32)          64000     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1250, 32)          3104      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 625, 32)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               5000250   
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 251       
Total params: 5,067,605
Trainable params: 5,067,605
Non-trainable params: 0
__________________________________

KeyboardInterrupt: 

In [2]:
# 원 핫 인코딩 처리된 X_test배열을 한번 확인 해 봅니다. (큰 의미는 없지만 추후 교차 검증을 거칠때 필요하기도 합니다..)

X_test[0][:100]

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1.])

In [23]:
#위에서 저장된 모델을 통해 X_test 값을 다시 넣어 보아, 정상적으로 작동하는지 확인합니다.

model = load_model('temp.h5')
model.summary()

a = model.predict(X_test)
k = 0
for i in a:
    k += i

print(k/len(a))

[0.8353868]
